<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0" width=150px> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Generative AI with OpenAI API</h1>
<h1>Embeddings</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from pprint import pprint

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt 

from ipywidgets import interact

import tiktoken
import openai

import nltk
from nltk.corpus import reuters
from nltk import bigrams, trigrams

import sklearn
from sklearn.metrics.pairwise import cosine_similarity

from openai.embeddings_utils import (
    get_embedding,
    distances_from_embeddings,
    tsne_components_from_embeddings,
    chart_from_components,
    indices_of_nearest_neighbors_from_distances,
)

import os
import gzip

import tqdm as tq
from tqdm.notebook import tqdm
tqdm.pandas()

import watermark

%load_ext watermark
%matplotlib inline

We start by printing out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.10.9
IPython version      : 8.10.0

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 23.1.0
Machine     : x86_64
Processor   : i386
CPU cores   : 16
Architecture: 64bit

Git hash: dd79ad5dfd349dc3b1ae00a307ee636a9ab6b2ef

pandas    : 1.5.3
openai    : 0.28.1
nltk      : 3.7
matplotlib: 3.7.2
tqdm      : 4.66.1
watermark : 2.4.2
tiktoken  : 0.5.1
json      : 2.0.9
sklearn   : 1.2.1
numpy     : 1.23.5



Load default figure style

In [3]:
plt.style.use('d4sci.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

Get the API key

In [4]:
openai.api_key = os.getenv("OPENAI_API_KEY")

# Extract embeddings

In [5]:
response = openai.Embedding.create(
    model="text-embedding-ada-002",
    input="Your text goes here",
    )

In [6]:
response

<OpenAIObject list at 0x7f866919f1a0> JSON: {
  "object": "list",
  "data": [
    {
      "object": "embedding",
      "index": 0,
      "embedding": [
        -0.013017537072300911,
        -0.013470551930367947,
        0.009679877199232578,
        -0.021624833345413208,
        -0.024995801970362663,
        0.04375597834587097,
        -0.019586263224482536,
        -0.015842221677303314,
        -0.010779106989502907,
        -0.023383600637316704,
        0.005276299547404051,
        0.016628338024020195,
        -0.009846427477896214,
        -0.0009176897001452744,
        0.0031561197247356176,
        -0.006288922391831875,
        0.024542786180973053,
        -0.003993865568190813,
        0.014429879374802113,
        -0.012331351637840271,
        -0.004580121021717787,
        0.01320407260209322,
        0.013217397034168243,
        0.012377985753118992,
        -0.004836607724428177,
        0.0011800057254731655,
        0.011705123819410801,
        -0.01729453727

The embedding itself is a high dimensional vector

In [7]:
len(response["data"][0]["embedding"])

1536

And the similarity between vectors represents the similarity between the texts

In [8]:
emb1 = openai.Embedding.create(
    model="text-embedding-ada-002",
    input="love",
    )["data"][0]["embedding"]

emb2 = openai.Embedding.create(
    model="text-embedding-ada-002",
    input="kindness",
    )["data"][0]["embedding"]

Cosine similarity is a quick and dirty way to measure how similar two vectors are

In [9]:
cosine_similarity([emb1], [emb2])

array([[0.83823629]])

Naturally, unrelated words have much lower degree of similarity

In [10]:
emb3 = openai.Embedding.create(
    model="text-embedding-ada-002",
    input="airplane",
    )["data"][0]["embedding"]

In [11]:
cosine_similarity([emb2], [emb3])

array([[0.77027252]])

# Generate embeddings for the Reuters sentences dataset

We're going to use Reuters

In [12]:
sentences = [" ".join(word for word in sent) for sent in reuters.sents()]

In [13]:
len(sentences)

54711

In [14]:
sentences[:10]

["ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPAN RIFT Mounting trade friction between the U . S . And Japan has raised fears among many of Asia ' s exporting nations that the row could inflict far - reaching economic damage , businessmen and officials said .",
 'They told Reuter correspondents in Asian capitals a U . S . Move against Japan might boost protectionist sentiment in the U . S . And lead to curbs on American imports of their products .',
 "But some exporters said that while the conflict would hurt them in the long - run , in the short - term Tokyo ' s loss might be their gain .",
 "The U . S . Has said it will impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17 , in retaliation for Japan ' s alleged failure to stick to a pact not to sell semiconductors on world markets at below cost .",
 'Unofficial Japanese estimates put the impact of the tariffs at 10 billion dlrs and spokesmen for major electronics firms said they would virtually halt expo

Let us build a DataFrame of 1000 sentences

In [15]:
data = pd.DataFrame(sentences[:1000], columns=["text"])

In [16]:
data

,text
0,ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPA...
1,They told Reuter correspondents in Asian capit...
2,But some exporters said that while the conflic...
3,The U . S . Has said it will impose 300 mln dl...
4,Unofficial Japanese estimates put the impact o...
...,...
995,The decision to cut deposits was taken by the ...
996,The cuts were likely to attract more business ...
997,ELECTRO RENT CORP & lt ; ELRC > 3RD QTR FEB 28...
998,WALGREEN CO 2ND QTR SHR 62 CTS VS 58 CTS


In [17]:
embedding_model = "text-embedding-ada-002"

In [18]:
encoding = tiktoken.encoding_for_model(embedding_model)

# Compute the embeddings for each sentence

In [19]:
data["embedding"] = data.text.progress_apply(lambda x: get_embedding(x, engine=embedding_model))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [20]:
data

,text,embedding
0,ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPA...,"[-0.016210949048399925, -0.028212646022439003,..."
1,They told Reuter correspondents in Asian capit...,"[-0.021910814568400383, -0.016312869265675545,..."
2,But some exporters said that while the conflic...,"[-0.020641576498746872, -0.018287593498826027,..."
3,The U . S . Has said it will impose 300 mln dl...,"[-0.01981060765683651, -0.01667136885225773, 0..."
4,Unofficial Japanese estimates put the impact o...,"[-0.021219488233327866, -0.03354521840810776, ..."
...,...,...
995,The decision to cut deposits was taken by the ...,"[0.003903309814631939, -0.014889535494148731, ..."
996,The cuts were likely to attract more business ...,"[-0.020308012142777443, -0.02054002694785595, ..."
997,ELECTRO RENT CORP & lt ; ELRC > 3RD QTR FEB 28...,"[0.0014385030372068286, -0.012292816303670406,..."
998,WALGREEN CO 2ND QTR SHR 62 CTS VS 58 CTS,"[0.0023896058555692434, 0.006466805934906006, ..."


# Generate Recommendations

In [21]:
def get_recommendations(data, pos, k_nearest_neighbors=3):
    embeddings = np.array(data["embedding"].to_list())
    strings = data["text"].to_list()

    query_embedding = embeddings[pos]

    distances = distances_from_embeddings(query_embedding, embeddings, 
                                          distance_metric="cosine")

    indices_of_nearest_neighbors = indices_of_nearest_neighbors_from_distances(distances)

    query_string = strings[pos]
    print(f"Query: {query_string}\n\n")

    for pos, i in enumerate(indices_of_nearest_neighbors[1:k_nearest_neighbors+1]):
        print("-- Nearest neighbor %u of %u (%u)---\n%s\n\n" % (pos, k_nearest_neighbors, i, strings[i]))

In [22]:
get_recommendations(data, 123, 5)

Query: It will be the next decade before we see if the strategy is right or wrong ."


-- Nearest neighbor 0 of 5 (94)---
But financial analysts are divided on whether and how quickly the gamble will pay off .


-- Nearest neighbor 1 of 5 (118)---
We expect this will be allowed in two or three years ," he said .


-- Nearest neighbor 2 of 5 (106)---
We only have to wait two or three years , not until the 21st century ," Komatsu said .


-- Nearest neighbor 3 of 5 (321)---
Bankers say it is too early to venture a forecast for economic growth this year or next .


-- Nearest neighbor 4 of 5 (580)---
" And we will have to see if the United States is able to do what they promised in Paris on reducing the budget deficit -- and get it through Congress ," he added .




<center>
     <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>